# Ottimizzazione dello stock

In questo notebook esploriamo una strategia di gestione della supply chain focalizzata sulla prevenzione della rottura di stock tramite il calcolo dinamico degli ordini. 

Illustriamo come, a partire da previsioni di vendita e parametri come stock iniziale e lead time, sia possibile simulare l’evoluzione del magazzino e programmare più ordini nel tempo per evitare esaurimenti.

Estendiamo il modello per analizzare diversi livelli di servizio e introduciamo una simulazione con ordini multipli che tiene conto del lead time e degli ordini già in arrivo.

Infine, affrontiamo il tema della gestione multi-magazzino, mostrando come la rappresentazione della rete di magazzini tramite grafi consenta di modellare efficacemente le relazioni tra nodi (magazzini) e ottimizzare spostamenti di merce.

In [ ]:
import datetime

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
from scipy.stats import norm

## Parametri di simulazione

Impostiamo i parametri principali della simulazione:

- orizzonte temporale (in giorni),
- stock iniziale disponibile,
- tempo di consegna del fornitore (lead time),
- livelli di servizio da confrontare.

Le vendite giornaliere sono simulate come variabili casuali secondo una distribuzione di Poisson.

In [ ]:
np.random.seed(42)

giorni = 60
stock_iniziale = 200
lead_time = 4
livelli_servizio = [0.8, 0.9, 0.95, 0.99]

vendite_giornaliere = np.random.poisson(20, size=giorni)
date = pd.date_range(start=datetime.datetime.now(), periods=giorni)
df = pd.DataFrame({"data": date, "vendita_prevista": vendite_giornaliere})

## Calcolo del livello di riordino per diversi livelli di servizio

Il livello di riordino viene definito come:

$
l_r = \mu + z \cdot \sigma
$

dove:

- $\mu$: domanda media durante il lead time,
- $\sigma$: deviazione standard della domanda durante il lead time,
- $z$: quantile corrispondente al livello di servizio desiderato.


In [ ]:
media_domanda = np.mean(vendite_giornaliere[:lead_time])
std_domanda = np.std(vendite_giornaliere[:lead_time])

livelli = []
for sl in livelli_servizio:
    z = norm.ppf(sl)
    safety_stock = z * std_domanda
    livello_riordino = int(media_domanda + safety_stock)
    livelli.append((sl, livello_riordino))


In [ ]:
for sl, lr in livelli:
    print(f"Service Level {sl:.0%} → Livello di Riordino: {lr}")

## Strategia di riordino

Simuliamo ora lo stock lungo l’orizzonte temporale introducendo:

- una soglia dinamica di riordino calcolata a partire dal livello di servizio selezionato (es. 95%),
- un algoritmo che effettua un nuovo ordine ogni volta che lo stock scende sotto tale soglia,
- il reintegro dello stock dopo il lead time.

Questo riflette un comportamento ricorrente e realistico nel tempo.

In [ ]:
# Simulazione dello stock con più ordini e lead time
qta_ordine = stock_iniziale
ordini_effettuati = []

df["stock_con_ordine"] = stock_iniziale
scorte_ordinate = 0

for i in range(1, len(df)):
    df.loc[i, "stock_con_ordine"] = df.loc[i-1, "stock_con_ordine"] - df.loc[i, "vendita_prevista"]

    # Considera ordini già arrivati
    if i >= lead_time:
        df.loc[i, "stock_con_ordine"] += ordini_effettuati.count(i - lead_time) * qta_ordine

    # Calcolo domanda attesa nel lead time a partire da oggi
    domanda_attesa = df.loc[i:i+lead_time-1, "vendita_prevista"].sum() if i + lead_time - 1 < len(df) else df.loc[i:, "vendita_prevista"].sum()

    safety_stock = int(norm.ppf(service_level) * np.std(df.loc[i:i+lead_time-1, "vendita_prevista"])) if i + lead_time - 1 < len(df) else int(norm.ppf(service_level) * np.std(df.loc[i:, "vendita_prevista"]))
    livello_riordino = domanda_attesa + safety_stock

    # Se lo stock attuale più ordini in arrivo non basta per coprire la domanda nel lead time, effettua ordine
    stock_disponibile = df.loc[i, "stock_con_ordine"] + scorte_ordinate
    if stock_disponibile <= livello_riordino:
        ordini_effettuati.append(i-1)
        scorte_ordinate += qta_ordine

    # Rimuovi ordini arrivati dallo stock ordinato in attesa
    if i - lead_time in ordini_effettuati:
        scorte_ordinate -= qta_ordine


In [ ]:
# Estrazione date di ordini e arrivi per visualizzazione
date_ordini = [df.loc[idx, "data"] for idx in ordini_effettuati]
date_arrivi = [d + pd.Timedelta(days=lead_time) for d in date_ordini]

# Grafico con evidenziazione ordini e arrivi
plt.figure(figsize=(12, 6))
# plt.plot(df["data"], df["stock"], label="Stock senza ordini")
plt.plot(df["data"], df["stock_con_ordine"], label="Stock")
plt.axhline(livello_riordino, color="orange", linestyle="--", label="Livello di Riordino")
plt.axhline(0, color="red", linestyle="--")

plt.xlabel("Data")
plt.ylabel("Stock")
plt.title("Andamento dello Stock")

for d_ordine, d_arrivo in zip(date_ordini, date_arrivi):
    plt.axvline(d_ordine, color="green", linestyle="--", alpha=0.7, label="Data Ordine")
    plt.axvline(d_arrivo, color="violet", linestyle=":", alpha=0.7, label="Arrivo Ordine")

handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.tight_layout()
plt.show()


## Estensione multi-magazzino

In questa sezione estendiamo la simulazione per gestire più magazzini indipendenti.  
Ogni magazzino ha caratteristiche proprie: stock iniziale, capacità, lead time, livelli di servizio e domanda assegnata.  
La domanda giornaliera viene ripartita tra i magazzini in modo uniforme (per semplicità).

L’obiettivo è simulare la gestione indipendente degli stock e degli ordini per ciascun magazzino, monitorando lo stock residuo e gli ordini piazzati.

In [ ]:
giorni = 60
num_magazzini = 3

# Parametri diversi per ciascun magazzino
stock_iniziali = [50, 30, 40]
capacita = [100, 60, 80]
lead_times = [3, 2, 4]
livelli_servizio = [0.95, 0.9, 0.99]
z_scores = [1.645, 1.28, 2.33]  # corrispondenti a livelli servizio

vendite_giornaliere_tot = np.random.poisson(15, size=giorni)
date = pd.date_range(start=datetime.datetime.now(), periods=giorni)

# Suddivido domanda uniformemente (per semplicità)
vendite_per_magazzino = [vendite_giornaliere_tot // num_magazzini] * num_magazzini

# Creo dizionari per tenere dati di ogni magazzino
magazzini = {}

for i in range(num_magazzini):
    magazzini[i] = {
        "stock": [stock_iniziali[i]],
        "ordini": [],
        "lead_time": lead_times[i],
        "z": z_scores[i],
        "livello_servizio": livelli_servizio[i],
        "capacita": capacita[i],
        "vendite": vendite_per_magazzino[i]
    }

for giorno in range(giorni):
    for i in range(num_magazzini):
        dati = magazzini[i]
        stock = dati["stock"][-1]
        
        # Calcolo domanda cumulata lead time
        end_lt = min(giorno + dati["lead_time"], giorni - 1)
        domanda_cumulata = dati["vendite"][giorno:end_lt+1].sum()
        
        # Deviazione std e safety stock (approssimazione semplice)
        deviazione_std = np.sqrt(np.var(dati["vendite"][giorno:end_lt+1]) * (end_lt - giorno + 1))
        safety_stock = dati["z"] * deviazione_std if not np.isnan(deviazione_std) else 0
        
        livello_riordino = domanda_cumulata + safety_stock
        
        # Ordini in arrivo
        ordini_in_arrivo = sum([qta for (giorno_arrivo, qta) in dati["ordini"] if giorno_arrivo > giorno])
        
        stock_previsionale = stock + ordini_in_arrivo - domanda_cumulata
        
        if stock_previsionale < livello_riordino:
            qta_da_ordinare = max(0, dati["capacita"] - (stock + ordini_in_arrivo))
            giorno_arrivo = giorno + dati["lead_time"]
            dati["ordini"].append((giorno_arrivo, qta_da_ordinare))
        else:
            qta_da_ordinare = 0
        
        # Aggiorno stock sottraendo domanda del giorno
        stock = stock - dati["vendite"][giorno]
        
        # Arrivo ordini oggi
        ordini_oggi = sum([qta for (giorno_arrivo, qta) in dati["ordini"] if giorno_arrivo == giorno])
        stock += ordini_oggi
        
        dati["stock"].append(stock)

# Preparo dataframe per il grafico
df_magazzini = pd.DataFrame({"data": date})

for i in range(num_magazzini):
    df_magazzini[f"stock_magazzino_{i}"] = magazzini[i]["stock"][1:]  # scarto valore iniziale doppio


In [ ]:
plt.figure(figsize=(14, 7))
for i in range(num_magazzini):
    plt.plot(df_magazzini["data"], df_magazzini[f"stock_magazzino_{i}"], label=f"Magazzino {i}")
plt.title("Simulazione Multi-Magazzino - Stock Giornaliero")
plt.xlabel("Data")
plt.ylabel("Stock")
plt.legend()
plt.grid(True)
plt.show()

## Gestione multi-magazzino e trasferimento merci

In scenari con più magazzini distribuiti, la gestione delle scorte diventa più complessa rispetto al singolo magazzino.

Non basta ordinare solo dal fornitore: è possibile spostare merci tra magazzini per ottimizzare le scorte, ridurre i costi e migliorare i livelli di servizio.

### Problemi da risolvere

- Evitare rotture di stock in ciascun magazzino
- Minimizzare i costi di ordine e di trasporto
- Considerare tempi diversi per ordini esterni e trasferimenti interni
- Coordinare i flussi tra magazzini geograficamente distribuiti

### Meccanismo di trasferimento tra magazzini

| Passo                         | Descrizione                                                       |
|------------------------------|-----------------------------------------------------------------|
| 1. Analisi dello stock locale | Verificare stock residuo e domanda prevista in ciascun magazzino |
| 2. Verifica disponibilità     | Controllare se altri magazzini hanno stock eccedente            |
| 3. Decisione                  | Pianificare trasferimento interno se stock sufficiente           |
| 4. Ordine esterno             | Se nessun magazzino può fornire stock, effettuare ordine esterno |


## Rappresentare la rete di magazzini come un grafo

Una rete di magazzini, con i relativi flussi di merce tra di essi, può essere modellata efficacemente come un grafo orientato, dove:

- **I nodi rappresentano i singoli magazzini**  
  Ogni nodo corrisponde a un punto di stoccaggio fisico o virtuale, con caratteristiche proprie come capacità, stock attuale e posizione geografica.

- **Gli archi rappresentano i collegamenti logistici**  
  Ogni arco diretto indica una possibile tratta di spostamento della merce da un magazzino a un altro, con attributi come quantità trasferibile, tempi di consegna e costi.

Questa rappresentazione consente di:

- Visualizzare in modo chiaro e intuitivo la struttura della rete distributiva.
- Analizzare i flussi di materiale e individuare colli di bottiglia o percorsi inefficienti.
- Pianificare strategie di riordino, riallocazione delle scorte e ottimizzazione dei trasporti.
- Integrare facilmente algoritmi di ottimizzazione, simulazione e analisi di scenario.

In sintesi, il grafo fornisce una mappa dinamica e flessibile della rete logistica, indispensabile per decisioni operative e strategiche più consapevoli ed efficaci.

In [ ]:
G = nx.DiGraph()

# Definizione nodi (magazzini)
magazzini = ["Magazzino A", "Magazzino B", "Magazzino C"]
G.add_nodes_from(magazzini)

# Definizione archi (spostamenti)
# (origine, destinazione, peso)
spostamenti = [
    ("Magazzino A", "Magazzino B", 10),
    ("Magazzino B", "Magazzino C", 5),
    ("Magazzino C", "Magazzino A", 8),
    ("Magazzino A", "Magazzino C", 2)
]

for u, v, peso in spostamenti:
    G.add_edge(u, v, weight=peso)

pos = nx.spring_layout(G, seed=42)  # disposizione nodi

plt.figure(figsize=(8, 6))
nx.draw(G, pos, with_labels=True, node_size=3000, node_color="lightblue", font_size=12, font_weight="bold", arrowsize=20)

# Disegna le etichette dei pesi sugli archi
edge_labels = {(u, v): f'{d["weight"]}' for u, v, d in G.edges(data=True)}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='red', font_size=10)

plt.title("Rete di magazzini e spostamenti di merce")
plt.axis("off")
plt.show()

## Algoritmi di ottimizzazione che sfruttano la struttura a grafo

La rappresentazione della rete di magazzini come grafo apre la strada all’applicazione di diversi algoritmi di ottimizzazione avanzati, tra cui:

- **Algoritmi di percorso minimo (Shortest Path)**  
  Utilizzati per trovare il percorso più rapido o meno costoso per spostare merci tra magazzini, minimizzando tempi di consegna o costi di trasporto.

- **Flusso massimo (Maximum Flow)**  
  Permette di determinare la massima quantità di merce trasferibile simultaneamente attraverso la rete senza superare le capacità dei collegamenti, utile per bilanciare carichi e prevenire congestioni.

- **Algoritmi di routing dei veicoli (Vehicle Routing Problem, VRP)**  
  Ottimizzano i percorsi di veicoli che devono effettuare consegne o spostamenti tra più magazzini, minimizzando costi e tempi complessivi.

- **Programmazione lineare su reti (Network Linear Programming)**  
  Per ottimizzare decisioni di riordino e riallocazione, massimizzando il servizio clienti o minimizzando i costi totali, tenendo conto di capacità, lead time e domanda.

Questi algoritmi possono essere combinati e adattati in funzione degli obiettivi specifici di supply chain, garantendo un utilizzo efficiente delle risorse e una gestione proattiva degli stock.

# Conclusioni

Abbiamo dimostrato come un modello relativamente semplice, basato su previsioni di vendita e regole di riordino con lead time, possa essere potenziato per gestire scenari complessi come ordini multipli e multi-magazzino.

L’introduzione della rete di magazzini come grafo ha permesso di visualizzare e comprendere le connessioni logistiche e ha aperto la porta all’applicazione di algoritmi avanzati di ottimizzazione.

Tuttavia, la complessità reale di una supply chain richiede ulteriori approfondimenti e implementazioni che tengano conto di variabili aggiuntive quali capacità di stoccaggio, costi di trasporto, vincoli contrattuali e dinamiche di domanda più complesse.

Questa base didattica è un primo passo verso la costruzione di soluzioni integrate e adattabili, utili sia per aziende di medie dimensioni sia per scenari industriali più articolati.